In [64]:
import pandas as pd
from pandas import json_normalize

In [65]:
# Load the JSON file as a DataFrame
property_data_df = pd.read_json("Data/random_property_data.json")

In [66]:
print(property_data_df.columns)

Index(['abbreviatedAddress', 'address', 'apartmentsForRentInZipcodeSearchUrl',
       'attributionInfo', 'bathrooms', 'bedrooms', 'big', 'boroughId',
       'boroughSearchUrl', 'brokerId',
       ...
       'zestimateHighPercent', 'zestimateLowPercent', 'zillowOfferMarket',
       'zipPlusFour', 'zipcode', 'zipcodeSearchUrl', 'zoContactSubtitle',
       'zoMarketName', 'zoResaleStartAnOfferEnabled', 'zpid'],
      dtype='object', length=212)


In [67]:
desired_columns = [
    'address', 'homeType', 'livingArea', 'livingAreaUnits', 'lotSize', 'lotAreaUnits',
    'neighborhoodRegion', 'resoFacts', 'price', 'hiResImageLink', 'zpid'
]

filtered_property_data_df = property_data_df[desired_columns]

print(filtered_property_data_df.head())

                                             address       homeType   
0  {'city': 'Ames', 'community': None, 'neighborh...  SINGLE_FAMILY  \
1  {'city': 'Ames', 'community': None, 'neighborh...  SINGLE_FAMILY   
2  {'city': 'Ames', 'community': None, 'neighborh...            LOT   
3  {'city': 'Ames', 'community': None, 'neighborh...  SINGLE_FAMILY   
4  {'city': 'Ames', 'community': None, 'neighborh...  SINGLE_FAMILY   

   livingArea livingAreaUnits  lotSize lotAreaUnits   
0      1404.0     Square Feet     4330  Square Feet  \
1      2021.0     Square Feet     8001  Square Feet   
2         NaN            None    87120        Acres   
3      2215.0     Square Feet    49658        Acres   
4      1050.0     Square Feet     8702  Square Feet   

                   neighborhoodRegion   
0           {'name': 'Oak-Riverside'}  \
1        {'name': 'Somerset Village'}   
2                                None   
3                                None   
4  {'name': 'College Creek/Old Ames'}

In [68]:
# Flatten the nested data in the DataFrame
flattened_property_data_df = json_normalize(filtered_property_data_df.to_dict(orient="records"))
print(flattened_property_data_df.head())

        homeType  livingArea livingAreaUnits  lotSize lotAreaUnits   price   
0  SINGLE_FAMILY      1404.0     Square Feet     4330  Square Feet  239900  \
1  SINGLE_FAMILY      2021.0     Square Feet     8001  Square Feet  405000   
2            LOT         NaN            None    87120        Acres  228000   
3  SINGLE_FAMILY      2215.0     Square Feet    49658        Acres  924900   
4  SINGLE_FAMILY      1050.0     Square Feet     8702  Square Feet  470000   

                                      hiResImageLink        zpid address.city   
0  https://photos.zillowstatic.com/fp/df0bd099740...   115842056         Ames  \
1  https://photos.zillowstatic.com/fp/4f94778f62c...    93952548         Ames   
2  https://photos.zillowstatic.com/fp/17cec68ca0e...  2090796515         Ames   
3  https://photos.zillowstatic.com/fp/17839d6b2ae...   117852871         Ames   
4  https://photos.zillowstatic.com/fp/737c4d70a7b...   115844692         Ames   

  address.community  ... resoFacts.waterView

In [69]:
print(flattened_property_data_df['resoFacts.atAGlanceFacts'])

0     [{'factLabel': 'Type', 'factValue': 'Single Fa...
1     [{'factLabel': 'Type', 'factValue': 'Single Fa...
2                                                  None
3     [{'factLabel': 'Type', 'factValue': 'Single Fa...
4     [{'factLabel': 'Type', 'factValue': 'Single Fa...
5     [{'factLabel': 'Type', 'factValue': 'Townhouse...
6     [{'factLabel': 'Type', 'factValue': 'Single Fa...
7     [{'factLabel': 'Type', 'factValue': 'Single Fa...
8     [{'factLabel': 'Type', 'factValue': 'Townhouse...
9     [{'factLabel': 'Type', 'factValue': 'Townhouse...
10    [{'factLabel': 'Type', 'factValue': 'Single Fa...
11    [{'factLabel': 'Type', 'factValue': 'Single Fa...
12    [{'factLabel': 'Type', 'factValue': 'Townhouse...
13    [{'factLabel': 'Type', 'factValue': 'Single Fa...
14    [{'factLabel': 'Type', 'factValue': 'Single Fa...
15    [{'factLabel': 'Type', 'factValue': 'Single Fa...
16                                                 None
17    [{'factLabel': 'Type', 'factValue': 'Singl

In [70]:
atAGlanceFacts = flattened_property_data_df['resoFacts.atAGlanceFacts']

# Find unique factLabels to create the columns
fact_labels = set()
for row in atAGlanceFacts:
    if row:
        for fact in row:
            fact_labels.add(fact['factLabel'])

fact_labels = list(fact_labels)
facts_df = pd.DataFrame(columns=fact_labels)

for idx, row in atAGlanceFacts.items():
    if row:
        fact_dict = {}
        for fact in row:
            fact_label = fact['factLabel']
            fact_value = fact['factValue']
            fact_dict[fact_label] = fact_value
        facts_df = pd.concat([facts_df, pd.DataFrame(fact_dict, index=[idx])])
    else:
        facts_df = pd.concat([facts_df, pd.DataFrame({}, index=[idx])])

print(facts_df)

               HOA Year Built Offer Review Date                   Parking   
0              NaN       1958              None   1 Attached Garage space  \
1              NaN       2002              None  2 Attached Garage spaces   
2              NaN        NaN              None                       NaN   
3     $175 monthly       2010              None  3 Attached Garage spaces   
4              NaN       1895              None                      None   
5   $1,728 monthly       1986              None                      None   
6     $50 annually       1987              None  2 Attached Garage spaces   
7    $200 annually       2022              None  3 Attached Garage spaces   
8   $2,231 monthly       1997              None   1 Attached Garage space   
9     $202 monthly       2022              None  2 Attached Garage spaces   
10   $200 annually       2022              None  3 Attached Garage spaces   
11    $260 monthly       2021              None  3 Attached Garage spaces   

In [71]:
print(flattened_property_data_df['resoFacts.basement'])

0                                          Full
1                                      Finished
2                                          None
3                                Full,Sump Pump
4                       Full,Partial,Unfinished
5                                          None
6                               Walk-Out Access
7                     Full,Unfinished,Sump Pump
8                                Full,Sump Pump
9     Full,Unfinished,Walk-Out Access,Sump Pump
10           Daylight,Full,Unfinished,Sump Pump
11                               None,Sump Pump
12                             None,Crawl Space
13                               Full,Sump Pump
14                                         Full
15                                         Full
16                                         None
17                    Full,Unfinished,Sump Pump
18                                         None
19                   Unfinished,Walk-Out Access
Name: resoFacts.basement, dtype: object


In [72]:
facts_df['GarageType'] = facts_df['Parking'].apply(
    lambda x: "NA" if pd.isnull(x) or x == "None" or x == "NaN" else 'Attchd' if "Attached" in x else "BuiltIn"
)

In [73]:
facts_df = facts_df.drop(columns=[
    'Type', 'Heating', 'Cooling', 'Parking', 'HOA', 'Lot', 'Days on Zillow', 'Price/sqft', 'Buyers Agency Fee', 'Offer Review Date'
])

In [74]:
flattened_property_data_df = pd.concat([flattened_property_data_df, facts_df], axis=1)

In [75]:
flattened_property_data_df['numStories'] = flattened_property_data_df['resoFacts.stories'].apply(lambda x: "2Story" if x == 2 else "1Story" if x == 1 else None)
flattened_property_data_df['BldgType'] = flattened_property_data_df['homeType'].apply(
    lambda x: '1Fam' if x == "SINGLE_FAMILY" else "TwnhsE" if x == 'CONDO' or x == 'TOWNHOUSE' else "2fmCon" if x == "MULTI_FAMILY" else None
)


0     1Story
1     2Story
2       None
3     1Story
4     2Story
5     1Story
6     1Story
7     1Story
8     1Story
9     1Story
10    1Story
11    1Story
12    1Story
13    1Story
14    2Story
15      None
16      None
17    1Story
18      None
19    2Story
Name: numStories, dtype: object


In [76]:
flattened_property_data_df['FullBath'] = flattened_property_data_df['resoFacts.bathroomsFull'].fillna(0) + flattened_property_data_df['resoFacts.bathroomsThreeQuarter'].fillna(0)
flattened_property_data_df['HalfBath'] = flattened_property_data_df['resoFacts.bathroomsHalf'].fillna(0) + flattened_property_data_df['resoFacts.bathroomsOneQuarter'].fillna(0) + flattened_property_data_df['resoFacts.bathroomsPartial'].fillna(0)
flattened_property_data_df['BedroomAbvGr'] = flattened_property_data_df['resoFacts.bedrooms'].fillna(0)

In [77]:
flattened_property_data_df = flattened_property_data_df.drop(columns=[
    'livingAreaUnits', 'lotAreaUnits', 'neighborhoodRegion', 'neighborhoodRegion.name', 
    'resoFacts.aboveGradeFinishedArea','resoFacts.architecturalStyle', 'resoFacts.accessibilityFeatures', 'resoFacts.additionalParcelsDescription',
    'resoFacts.allowedPets', 'resoFacts.appliances', 'resoFacts.associationAmenities', 'resoFacts.associationFee', 'resoFacts.associationFee2',
    'resoFacts.associationFeeIncludes', 'resoFacts.associationName', 'resoFacts.associationName2', 'resoFacts.associationPhone',
    'resoFacts.associationPhone2', 'resoFacts.atAGlanceFacts', 'resoFacts.attic', 'resoFacts.availabilityDate', 'resoFacts.basement', 'resoFacts.bathrooms', 
    'resoFacts.bathroomsFull', 'resoFacts.bathroomsHalf', 'resoFacts.bathroomsOneQuarter', 'resoFacts.bathroomsPartial', 'resoFacts.bathroomsThreeQuarter', 
    'resoFacts.basementYN', 'resoFacts.bedrooms', 
])

In [78]:
reso_facts_df = flattened_property_data_df.filter(regex=r'^resoFacts\.', axis=1)

In [79]:
flattened_property_data_df['Fireplaces'] = reso_facts_df['resoFacts.fireplaces'].fillna(0)

In [80]:
flattened_property_data_df['Foundation'] = reso_facts_df['resoFacts.foundationDetails'].apply(
    lambda x: 'Slab' if 'Slab' in x else 'PConc' if "Poured" in x else "BrkTil" if "Brick/Mortar" in x else "CBlock" if "Block" in x else None
)

In [81]:
reso_facts_df['resoFacts.zoningDescription']

0         Residential
1                None
2                None
3                 Res
4     Residential Med
5               F-PRD
6                  RL
7         Residential
8               F-PRD
9     FS-RL-Suburbn R
10        Residential
11    Planned Residen
12              F-PRD
13    Residential Low
14    Residential Low
15                Res
16    Suburban Res Lo
17              FS-RL
18                  R
19               None
Name: resoFacts.zoningDescription, dtype: object

In [82]:
flattened_property_data_df['PoolArea'] = 0.0
flattened_property_data_df['PoolQC'] = 'NA'
flattened_property_data_df['Fence'] = 'NA'

In [83]:
non_reso_facts_df = flattened_property_data_df.filter(regex=r'^(?!resoFacts\.)', axis=1)
non_reso_facts_df.drop(columns=['homeType', 'numStories'], inplace=True)
non_reso_facts_df.rename(columns={'livingArea': 'GrLivArea', 'lotSize': 'LotArea', 'price': 'ListingPrice', 'Year Built': 'YearBuilt'}, inplace=True)
# Display the non-resoFacts DataFrame
print(non_reso_facts_df.head())

   GrLivArea  LotArea  ListingPrice   
0     1404.0     4330        239900  \
1     2021.0     8001        405000   
2        NaN    87120        228000   
3     2215.0    49658        924900   
4     1050.0     8702        470000   

                                      hiResImageLink        zpid address.city   
0  https://photos.zillowstatic.com/fp/df0bd099740...   115842056         Ames  \
1  https://photos.zillowstatic.com/fp/4f94778f62c...    93952548         Ames   
2  https://photos.zillowstatic.com/fp/17cec68ca0e...  2090796515         Ames   
3  https://photos.zillowstatic.com/fp/17839d6b2ae...   117852871         Ames   
4  https://photos.zillowstatic.com/fp/737c4d70a7b...   115844692         Ames   

  address.community address.neighborhood address.state address.streetAddress   
0              None                 None            IA     108 N Russell Ave  \
1              None                 None            IA      2739 Somerset Dr   
2              None                 No

C:\Users\chenb\AppData\Local\Temp\ipykernel_30784\499274730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_reso_facts_df.drop(columns=['homeType', 'numStories'], inplace=True)
C:\Users\chenb\AppData\Local\Temp\ipykernel_30784\499274730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_reso_facts_df.rename(columns={'livingArea': 'GrLivArea', 'lotSize': 'LotArea', 'price': 'ListingPrice', 'Year Built': 'YearBuilt'}, inplace=True)


In [84]:
print(non_reso_facts_df.columns)

Index(['GrLivArea', 'LotArea', 'ListingPrice', 'hiResImageLink', 'zpid',
       'address.city', 'address.community', 'address.neighborhood',
       'address.state', 'address.streetAddress', 'address.subdivision',
       'address.zipcode', 'YearBuilt', 'GarageType', 'BldgType', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'Fireplaces', 'Foundation', 'PoolArea',
       'PoolQC', 'Fence'],
      dtype='object')


In [90]:
df = non_reso_facts_df[non_reso_facts_df['BldgType'].notna()]

In [91]:
df.to_csv("Data/ames_iowa_detailed_listings.csv", index=False)
print("Data/ames_iowa_detailed_listings.csv")

Data/ames_iowa_detailed_listings.csv


In [96]:
import csv
import json

In [98]:
# create json for opensearch
indexed_data = []

for _, row in df.iterrows():
    indexed_data.append({
        'index': {
            '_index': 'listings',
            '_id': row['zpid']
        }
    })
    indexed_data.append({
        'zpid': row['zpid'],
        'city': row['address.city'],
        'state': row['address.state'],
        'zipcode': row['address.zipcode'],
        'GrLivArea': row['GrLivArea'],
        'LotArea': row['LotArea'],
        'ListingPrice': row['ListingPrice'],
        'FullBath': row['FullBath'],
        'HalfBath': row['HalfBath'],
        'BedroomAbvGr': row['BedroomAbvGr']
    })

with open('Data/ames_iowa_listings.json', mode='w', encoding='utf-8') as json_file:
    for item in indexed_data:
        json.dump(item, json_file)
        json_file.write('\n')